# NN Model Building on The Lion King (2019) Movie Reviews


### In this section
1. I have applied various neural network algorithmns and 1D CNN was the best.
2. I have built an SVM classifier with the output of the hidden layer.
3. I have built a model using Pretrained Embeddings from GLOVE.

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore','RuntimeWarning')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import regularizers
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import GlobalMaxPooling1D, AveragePooling1D,Conv1D,Dropout,Embedding, MaxPooling1D,GRU,Bidirectional,LSTM,SpatialDropout1D,GlobalMaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

pd.set_option('display.max_colwidth', -1)

C:\Users\lokesh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\lokesh\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lokesh\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lokesh\Anaconda3\lib\site-packages\tensorflow\python\framew

In [2]:
data1 = pd.read_csv('alreviews_df_3000.csv')
valid2 = pd.read_csv('alreviews_df_1100_validation.csv')
test = pd.read_csv('test-1566381431512.csv')

In [3]:
data1.drop(['createDate','displayImageUrl','displayName','hasProfanity','hasSpoilers','isSuperReviewer','isVerified','rating','timeFromCreation','updateDate','primary_key'],axis=1,inplace=True)
valid2.drop(['createDate','displayImageUrl','displayName','hasProfanity','hasSpoilers','isSuperReviewer','isVerified','rating','timeFromCreation','updateDate'],axis=1,inplace=True)

In [4]:
data1['sentiment'] = np.where((data1['score']>3.0),0,1)
valid2['sentiment'] = np.where((valid2['score']>3.0),0,1)

In [5]:
CONTRACTION_MAP = {"ain't": 'is not', "aren't": 'are not', "can't": 'cannot', "can't've": 'cannot have', "'cause": 'because', "could've": 'could have', "couldn't": 'could not', "couldn't've": 'could not have', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hadn't've": 'had not have', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'd've": 'he would have', "he'll": 'he will', "he'll've": 'he he will have', "he's": 'he is', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "I'd": 'I would', "I'd've": 'I would have', "I'll": 'I will', "I'll've": 'I will have', "I'm": 'I am', "I've": 'I have', "i'd": 'i would', "i'd've": 'i would have', "i'll": 'i will', "i'll've": 'i will have', "i'm": 'i am', "i've": 'i have', "isn't": 'is not', "it'd": 'it would', "it'd've": 'it would have', "it'll": 'it will', "it'll've": 'it will have', "it's": 'it is', "let's": 'let us', "ma'am": 'madam', "mayn't": 'may not', "might've": 'might have', "mightn't": 'might not', "mightn't've": 'might not have', "must've": 'must have', "mustn't": 'must not', "mustn't've": 'must not have', "needn't": 'need not', "needn't've": 'need not have', "o'clock": 'of the clock', "oughtn't": 'ought not', "oughtn't've": 'ought not have', "shan't": 'shall not', "sha'n't": 'shall not', "shan't've": 'shall not have', "she'd": 'she would', "she'd've": 'she would have', "she'll": 'she will', "she'll've": 'she will have', "she's": 'she is', "should've": 'should have', "shouldn't": 'should not', "shouldn't've": 'should not have', "so've": 'so have', "so's": 'so as', "that'd": 'that would', "that'd've": 'that would have', "that's": 'that is', "there'd": 'there would', "there'd've": 'there would have', "there's": 'there is', "they'd": 'they would', "they'd've": 'they would have', "they'll": 'they will', "they'll've": 'they will have', "they're": 'they are', "they've": 'they have', "to've": 'to have', "wasn't": 'was not', "we'd": 'we would', "we'd've": 'we would have', "we'll": 'we will', "we'll've": 'we will have', "we're": 'we are', "we've": 'we have', "weren't": 'were not', "what'll": 'what will', "what'll've": 'what will have', "what're": 'what are', "what's": 'what is', "what've": 'what have', "when's": 'when is', "when've": 'when have', "where'd": 'where did', "where's": 'where is', "where've": 'where have', "who'll": 'who will', "who'll've": 'who will have', "who's": 'who is', "who've": 'who have', "why's": 'why is', "why've": 'why have', "will've": 'will have', "won't": 'will not', "won't've": 'will not have', "would've": 'would have', "wouldn't": 'would not', "wouldn't've": 'would not have', "y'all": 'you all', "y'all'd": 'you all would', "y'all'd've": 'you all would have', "y'all're": 'you all are', "y'all've": 'you all have', "you'd": 'you would', "you'd've": 'you would have', "you'll": 'you will', "you'll've": 'you will have', "you're": 'you are', "you've": 'you have'}

In [6]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    #re.compile(regex).search(subject) is equivalent to re.search(regex, subject).
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
            if contraction_mapping.get(match)\
            else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = re.sub("’", "'", text)
    expanded_text = contractions_pattern.sub(expand_match, expanded_text)

    return expanded_text

In [7]:
# Function to Preprocess the Reviews
def clean_doc(doc):
    # Removing contractions
    doc = expand_contractions(doc)
    
    # split into tokens by white space
    tokens = doc.split(' ')
    
    # Converting into lower case
    tokens = [w.lower() for w in tokens]
    
    # remove special characters from each token
    tokens = [re.sub(r"[^a-zA-Z#\s]",'',i) for i in tokens]
    tokens = [re.sub(r"[\r\n]",'',i) for i in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    
    # lemmatizing
    lmtzr = nltk.stem.WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(w) for w in tokens]
    
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [8]:
data1.review = data1.review.apply(lambda x: ' '.join(clean_doc(x)))
valid2.review = valid2.review.apply(lambda x: ' '.join(clean_doc(x)))
test.Review = test.Review.apply(lambda x: ' '.join(clean_doc(x)))

In [9]:
X = data1.review
y = data1.sentiment

X_train, X_valid1, y_train, y_valid1 = train_test_split(X,y,test_size=0.3, random_state=1234)

In [10]:
X_valid2 = valid2.review
y_valid2 = valid2.sentiment

In [11]:
X_test = test.Review

In [12]:
max_num_words = 10000
seq_len = 50
embedding_size = 100

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_num_words) #Tokenizer is used to tokenize text
tokenizer.fit_on_texts(X_train) #Fit this to our corpus

X_train = tokenizer.texts_to_sequences(X_train) #'text to sequences converts the text to a list of indices
X_train = pad_sequences(X_train, maxlen=50) #pad_sequences makes every sequence a fixed size list by padding with 0s 
X_valid1 = tokenizer.texts_to_sequences(X_valid1) 
X_valid1 = pad_sequences(X_valid1, maxlen=50)
X_valid2 = tokenizer.texts_to_sequences(X_valid2) 
X_valid2 = pad_sequences(X_valid2, maxlen=50)
X_test = tokenizer.texts_to_sequences(X_test) 
X_test = pad_sequences(X_test, maxlen=50)

X_train.shape, X_valid1.shape, X_valid2.shape, X_test.shape # Check the dimensions of x_train and x_test  

((2100, 50), (900, 50), (1100, 50), (1200, 50))

In [14]:
from keras import backend as K

def Recall_score(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def Precision_score(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def F1_score(y_true, y_pred):
    precision = Precision_score(y_true, y_pred)
    recall = Recall_score(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Deep Neural Network

In [15]:
# create the model
model = Sequential()
model.add(Embedding(input_dim = max_num_words,input_length = seq_len,output_dim = embedding_size))
model.add(Flatten())
model.add(Dropout(rate=0.9))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[F1_score])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                250050    
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 1,251,351
Trainable params: 1,251,351
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_valid1, y_valid1), epochs=20, batch_size=64, verbose=2,)

Train on 2100 samples, validate on 900 samples
Epoch 1/20
 - 1s - loss: 0.5765 - F1_score: 0.0554 - val_loss: 0.5603 - val_F1_score: 0.0403
Epoch 2/20
 - 1s - loss: 0.5568 - F1_score: 0.1169 - val_loss: 0.5474 - val_F1_score: 0.1263
Epoch 3/20
 - 1s - loss: 0.5426 - F1_score: 0.2536 - val_loss: 0.5312 - val_F1_score: 0.2274
Epoch 4/20
 - 1s - loss: 0.5040 - F1_score: 0.3920 - val_loss: 0.4964 - val_F1_score: 0.3270
Epoch 5/20
 - 1s - loss: 0.4336 - F1_score: 0.5499 - val_loss: 0.4515 - val_F1_score: 0.3650
Epoch 6/20
 - 1s - loss: 0.3555 - F1_score: 0.6934 - val_loss: 0.4429 - val_F1_score: 0.4038
Epoch 7/20
 - 1s - loss: 0.3099 - F1_score: 0.7333 - val_loss: 0.4333 - val_F1_score: 0.4602
Epoch 8/20
 - 1s - loss: 0.2655 - F1_score: 0.7882 - val_loss: 0.4888 - val_F1_score: 0.4002
Epoch 9/20
 - 1s - loss: 0.2632 - F1_score: 0.7858 - val_loss: 0.4383 - val_F1_score: 0.5286
Epoch 10/20
 - 1s - loss: 0.2345 - F1_score: 0.8295 - val_loss: 0.4739 - val_F1_score: 0.4901
Epoch 11/20
 - 1s - lo

In [17]:
train_classes = model.predict_classes(X_train)
valid1_classes = model.predict_classes(X_valid1)
valid2_classes = model.predict_classes(X_valid2)

In [18]:
print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

Train F1 Score : 0.97
Valid1 F1 Score : 0.56
Valid2 F1 Score : 0.55


## 1D CNN

In [19]:
# create the model
model = Sequential()
model.add(Embedding(input_dim = max_num_words,input_length = seq_len,output_dim = embedding_size))
model.add(Dropout(rate=0.5))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[F1_score])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           1000000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 32)            9632      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 32)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 250)               200250    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 251       
Total para

In [20]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_valid1, y_valid1), epochs=5, batch_size=128, verbose=2)

Train on 2100 samples, validate on 900 samples
Epoch 1/5
 - 62s - loss: 0.6100 - F1_score: 0.0254 - val_loss: 0.5927 - val_F1_score: 0.0000e+00
Epoch 2/5
 - 49s - loss: 0.5610 - F1_score: 0.0029 - val_loss: 0.5540 - val_F1_score: 0.0545
Epoch 3/5
 - 49s - loss: 0.5329 - F1_score: 0.2215 - val_loss: 0.5250 - val_F1_score: 0.3356
Epoch 4/5
 - 49s - loss: 0.4578 - F1_score: 0.3987 - val_loss: 0.4475 - val_F1_score: 0.3455
Epoch 5/5
 - 50s - loss: 0.3280 - F1_score: 0.7043 - val_loss: 0.4161 - val_F1_score: 0.5377


In [21]:
train_classes = model.predict_classes(X_train)
valid1_classes = model.predict_classes(X_valid1)
valid2_classes = model.predict_classes(X_valid2)

In [22]:
print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

Train F1 Score : 0.81
Valid1 F1 Score : 0.54
Valid2 F1 Score : 0.56


## 1D CNN

In [23]:
model = Sequential()
model.add(Embedding(input_dim = max_num_words,input_length = seq_len,output_dim = embedding_size))
model.add(Conv1D(64, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=[F1_score])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           1000000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 64)            19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 1,027,713
Trainable params: 1,027,713
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(X_train, y_train,
          batch_size=64,
          epochs=5,
          validation_data=(X_valid1, y_valid1),verbose=2)

Train on 2100 samples, validate on 900 samples
Epoch 1/5
 - 49s - loss: 0.5648 - F1_score: 0.0000e+00 - val_loss: 0.4968 - val_F1_score: 0.0065
Epoch 2/5
 - 49s - loss: 0.3961 - F1_score: 0.5033 - val_loss: 0.3893 - val_F1_score: 0.6402
Epoch 3/5
 - 49s - loss: 0.2647 - F1_score: 0.8084 - val_loss: 0.3696 - val_F1_score: 0.6864
Epoch 4/5
 - 49s - loss: 0.1841 - F1_score: 0.8791 - val_loss: 0.3910 - val_F1_score: 0.7005
Epoch 5/5
 - 50s - loss: 0.1232 - F1_score: 0.9252 - val_loss: 0.4382 - val_F1_score: 0.6100


In [26]:
train_classes = model.predict_classes(X_train)
valid1_classes = model.predict_classes(X_valid1)
valid2_classes = model.predict_classes(X_valid2)

In [27]:
print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

Train F1 Score : 0.96
Valid1 F1 Score : 0.62
Valid2 F1 Score : 0.65


In [28]:
# test_classes = model.predict_classes(X_test)

# submission = pd.read_csv('samplesubmission.csv')

# submission.sentiment = test_classes.astype('int64')

# submission.sentiment.dtype

# submission.sentiment.value_counts(normalize=True)
# Test Score- 70

## Using Hidden Layer output in SVM

In [29]:
import keras.backend as K
emd = K.function(inputs=[model.layers[0].input], 
                 outputs=[model.layers[0].output])

In [30]:
emm_train = emd([X_train])[0]
emm_valid1 = emd([X_valid1])[0]
emm_valid2 = emd([X_valid2])[0]

In [31]:
nsamples, nx, ny = emm_train.shape
emm_train = emm_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = emm_valid1.shape
emm_valid1 = emm_valid1.reshape((nsamples,nx*ny))

nsamples, nx, ny = emm_valid2.shape
emm_valid2 = emm_valid2.reshape((nsamples,nx*ny))

In [32]:
from sklearn import svm

In [33]:
svm_classifier_linear = svm.SVC(kernel="linear", class_weight="balanced",C=0.05)

svm_clf = svm_classifier_linear.fit(emm_train,y_train)

train_pred = svm_clf.predict(emm_train)

valid1_pred = svm_clf.predict(emm_valid1)

valid2_pred = svm_clf.predict(emm_valid2)

print('Train F1 Score :',round(f1_score(y_train,train_pred),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_pred),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_pred),2))

Train F1 Score : 0.88
Valid1 F1 Score : 0.65
Valid2 F1 Score : 0.68


## LSTM

In [34]:
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim = max_num_words, 
                    input_length = seq_len, 
                    output_dim = embedding_size)) # Add an embedding layer which represents each unique token as a vector
#model.add(Dropout(rate=0.5))
model.add(LSTM(20, return_sequences=True,dropout=0.9)) # Add an LSTM layer
model.add(LSTM(10, return_sequences=False))
#model.add(Dropout(rate=0.5))
model.add(Dense(1, activation='sigmoid'))

In [35]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=[F1_score])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 100)           1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 20)            9680      
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                1240      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
Total params: 1,010,931
Trainable params: 1,010,931
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.fit(X_train, y_train,
          batch_size=64,
          epochs=20,
          validation_data=(X_valid1, y_valid1),verbose=2)

Train on 2100 samples, validate on 900 samples
Epoch 1/20
 - 4s - loss: 0.5993 - F1_score: 0.0167 - val_loss: 0.5546 - val_F1_score: 0.0000e+00
Epoch 2/20
 - 2s - loss: 0.5561 - F1_score: 0.0000e+00 - val_loss: 0.5412 - val_F1_score: 0.0000e+00
Epoch 3/20
 - 2s - loss: 0.5376 - F1_score: 0.0419 - val_loss: 0.5224 - val_F1_score: 0.0290
Epoch 4/20
 - 2s - loss: 0.5150 - F1_score: 0.1635 - val_loss: 0.5000 - val_F1_score: 0.1791
Epoch 5/20
 - 2s - loss: 0.4918 - F1_score: 0.3668 - val_loss: 0.4721 - val_F1_score: 0.3020
Epoch 6/20
 - 2s - loss: 0.4419 - F1_score: 0.4994 - val_loss: 0.4382 - val_F1_score: 0.4555
Epoch 7/20
 - 2s - loss: 0.4097 - F1_score: 0.5903 - val_loss: 0.4240 - val_F1_score: 0.5045
Epoch 8/20
 - 2s - loss: 0.3905 - F1_score: 0.6474 - val_loss: 0.4211 - val_F1_score: 0.5215
Epoch 9/20
 - 2s - loss: 0.3571 - F1_score: 0.6723 - val_loss: 0.4208 - val_F1_score: 0.5581
Epoch 10/20
 - 2s - loss: 0.3459 - F1_score: 0.6883 - val_loss: 0.4188 - val_F1_score: 0.5975
Epoch 11/2

In [37]:
train_classes = model.predict_classes(X_train)
valid1_classes = model.predict_classes(X_valid1)
valid2_classes = model.predict_classes(X_valid2)

In [38]:
print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

Train F1 Score : 0.89
Valid1 F1 Score : 0.63
Valid2 F1 Score : 0.65


In [39]:
max_num_words,seq_len,embedding_size

(10000, 50, 100)

## GRU

In [40]:
model = Sequential()
model.add(Embedding(input_dim = max_num_words, 
                    input_length = seq_len, 
                    output_dim = 150))
model.add(GRU(32,dropout=0.8,recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 150)           1500000   
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                17568     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 1,517,601
Trainable params: 1,517,601
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=32,
          epochs=7,
          validation_data=(X_valid1, y_valid1),verbose=2)

Train on 2100 samples, validate on 900 samples
Epoch 1/7
 - 5s - loss: 0.6023 - acc: 0.7048 - val_loss: 0.5335 - val_acc: 0.7222
Epoch 2/7
 - 4s - loss: 0.4969 - acc: 0.7371 - val_loss: 0.4767 - val_acc: 0.7422
Epoch 3/7
 - 4s - loss: 0.4111 - acc: 0.8157 - val_loss: 0.4453 - val_acc: 0.7600
Epoch 4/7
 - 4s - loss: 0.3267 - acc: 0.8633 - val_loss: 0.4612 - val_acc: 0.7711
Epoch 5/7
 - 4s - loss: 0.2755 - acc: 0.8948 - val_loss: 0.5063 - val_acc: 0.7722
Epoch 6/7
 - 4s - loss: 0.2357 - acc: 0.9105 - val_loss: 0.5408 - val_acc: 0.7689
Epoch 7/7
 - 3s - loss: 0.2066 - acc: 0.9219 - val_loss: 0.5749 - val_acc: 0.7611


In [42]:
train_classes = model.predict_classes(X_train)
valid1_classes = model.predict_classes(X_valid1)
valid2_classes = model.predict_classes(X_valid2)

In [43]:
print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

Train F1 Score : 0.91
Valid1 F1 Score : 0.56
Valid2 F1 Score : 0.61


# Using Pretrained Embeddings from GLOVE

In [44]:
# embeddings_index = {}
# f = open('glove.6B.100d.txt',encoding='utf8')
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

In [45]:
# word_index = tokenizer.word_index

In [46]:
# embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector

In [47]:
# embedding_layer = Embedding(len(word_index) + 1,
#                             embedding_size,
#                             weights=[embedding_matrix],
#                             input_length=seq_len,
#                             trainable=False)

In [48]:
# model = Sequential()
# model.add(Embedding(len(word_index) + 1,
#                             embedding_size,
#                             weights=[embedding_matrix],
#                             input_length=seq_len,
#                             trainable=False))
# model.add(Conv1D(64, 3, activation='relu'))
# model.add(Conv1D(64, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))


In [49]:
# model.summary()

In [50]:
# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=[F1_score])
# model.fit(X_train, y_train,
#           batch_size=64,
#           epochs=5,
#           validation_data=(X_valid1, y_valid1),verbose=2)

In [51]:
# train_classes = model.predict_classes(X_train)
# valid1_classes = model.predict_classes(X_valid1)
# valid2_classes = model.predict_classes(X_valid2)

In [52]:
# print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
# print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
# print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

# GRU

layer 1 GRU(64,dropout=0.4,return_sequences=True)
layer 2 GRU(32,dropout=0.5,return_sequences=False)
batch_size=64, epochs=10,
Train F1 Score : 0.99
Valid1 F1 Score : 0.62
Valid2 F1 Score : 0.63
<br><br>
layer 1 GRU(64,dropout=0.9,return_sequences=True)
layer 2 GRU(32,dropout=0.5,return_sequences=False)
batch_size=64, epochs=10,
Train F1 Score : 0.9
Valid1 F1 Score : 0.62
Valid2 F1 Score : 0.65
<br><br>
layer 1 GRU(64,dropout=0.9,return_sequences=True)
layer 2 GRU(32,dropout=0.9,return_sequences=False)
batch_size=64, epochs=10,
Train F1 Score : 0.85
Valid1 F1 Score : 0.6
Valid2 F1 Score : 0.61
<br><br>
layer 1 GRU(64,dropout=0.9,return_sequences=True)
layer 2 GRU(32,dropout=0.9,return_sequences=False)
batch_size=16, epochs=10,
Train F1 Score : 0.9
Valid1 F1 Score : 0.67
Valid2 F1 Score : 0.69

clean_doc
Train F1 Score : 0.89
Valid1 F1 Score : 0.65
Valid2 F1 Score : 0.67
<br><br>
layer 1 GRU(64,dropout=0.5,return_sequences=True)
layer 2 GRU(32,dropout=0.5,return_sequences=False)
batch_size=16, epochs=10,
Train F1 Score : 0.99
Valid1 F1 Score : 0.63
Valid2 F1 Score : 0.64
<br><br>
layer 1 GRU(32,dropout=0.5,return_sequences=True)
layer 2 GRU(16,dropout=0.5,return_sequences=False)
batch_size=16, epochs=10,
Train F1 Score : 0.99
Valid1 F1 Score : 0.64
Valid2 F1 Score : 0.65
<br><br>
layer 1 GRU(32,dropout=0.9,return_sequences=True)
layer 2 GRU(16,dropout=0.9,return_sequences=False)
batch_size=16, epochs=10,
Train F1 Score : 0.88
Valid1 F1 Score : 0.65
Valid2 F1 Score : 0.65
<br><br>
layer 1 GRU(32,dropout=0.9,return_sequences=True)
layer 2 GRU(16,dropout=0.9,return_sequences=False)
batch_size=16, epochs=15,
Train F1 Score : 0.92
Valid1 F1 Score : 0.63
Valid2 F1 Score : 0.65
<br><br>
layer 1 GRU(32,dropout=0.9,return_sequences=True)
layer 2 GRU(16,dropout=0.9,return_sequences=False)
batch_size=16, epochs=10,
Train F1 Score : 0.89
Valid1 F1 Score : 0.62
Valid2 F1 Score : 0.67
<br><br>
layer 1 GRU(64,dropout=0.9,return_sequences=True)
layer 2 GRU(32,dropout=0.9,return_sequences=False)
batch_size=16, epochs=10,
Train F1 Score : 0.9
Valid1 F1 Score : 0.61
Valid2 F1 Score : 0.64
<br><br>
layer 1 GRU(64,dropout=0.9,return_sequences=True)
layer 2 GRU(32,dropout=0.9,return_sequences=False)
batch_size=8, epochs=10,
Train F1 Score : 0.91
Valid1 F1 Score : 0.63
Valid2 F1 Score : 0.67
<br><br>
layer 1 GRU(32,dropout=0.9,return_sequences=True)
layer 2 GRU(16,dropout=0.9,return_sequences=False)
batch_size=16, epochs=15,
Train F1 Score : 0.9
Valid1 F1 Score : 0.69
Valid2 F1 Score : 0.7
TEST SCORE : 0.7

In [53]:
model_GRU=Sequential()
model_GRU.add(Embedding(max_num_words,100,mask_zero=True))
model_GRU.add(GRU(32,dropout=0.9,return_sequences=True))
model_GRU.add(GRU(16,dropout=0.9,return_sequences=False))
model_GRU.add(Dense(1,activation='sigmoid'))

model_GRU.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
gru_2 (GRU)                  (None, None, 32)          12768     
_________________________________________________________________
gru_3 (GRU)                  (None, 16)                2352      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 17        
Total params: 1,015,137
Trainable params: 1,015,137
Non-trainable params: 0
_________________________________________________________________


In [54]:
model_GRU.compile(loss='binary_crossentropy',optimizer=Adam(lr = 0.001),metrics=[F1_score])
model_GRU.fit(X_train, y_train,batch_size=16,
                            epochs=12,
                            validation_data=(X_valid1, y_valid1),
                            verbose=2)

Train on 2100 samples, validate on 900 samples
Epoch 1/12
 - 11s - loss: 0.6294 - F1_score: 0.0403 - val_loss: 0.5973 - val_F1_score: 0.0000e+00
Epoch 2/12
 - 9s - loss: 0.5969 - F1_score: 0.0069 - val_loss: 0.5834 - val_F1_score: 0.0000e+00
Epoch 3/12
 - 10s - loss: 0.5752 - F1_score: 0.0294 - val_loss: 0.5458 - val_F1_score: 0.0000e+00
Epoch 4/12
 - 10s - loss: 0.5146 - F1_score: 0.2878 - val_loss: 0.4352 - val_F1_score: 0.3906
Epoch 5/12
 - 11s - loss: 0.4490 - F1_score: 0.4939 - val_loss: 0.4176 - val_F1_score: 0.6046
Epoch 6/12
 - 11s - loss: 0.3958 - F1_score: 0.6128 - val_loss: 0.4149 - val_F1_score: 0.5545
Epoch 7/12
 - 11s - loss: 0.3642 - F1_score: 0.6509 - val_loss: 0.4308 - val_F1_score: 0.5749
Epoch 8/12
 - 11s - loss: 0.3503 - F1_score: 0.6972 - val_loss: 0.4676 - val_F1_score: 0.5353
Epoch 9/12
 - 10s - loss: 0.3372 - F1_score: 0.6756 - val_loss: 0.4761 - val_F1_score: 0.5470
Epoch 10/12
 - 11s - loss: 0.2926 - F1_score: 0.7563 - val_loss: 0.5310 - val_F1_score: 0.5381
E

In [55]:
train_classes = model_GRU.predict_classes(X_train)
valid1_classes = model_GRU.predict_classes(X_valid1)
valid2_classes = model_GRU.predict_classes(X_valid2)

In [56]:
print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

Train F1 Score : 0.9
Valid1 F1 Score : 0.61
Valid2 F1 Score : 0.62


In [57]:
# test_classes = model_GRU.predict_classes(X_test)
# # Train F1 Score : 0.9
# # Valid1 F1 Score : 0.69
# # Valid2 F1 Score : 0.7
# Test score: 70

In [58]:
# submission = pd.read_csv('samplesubmission.csv')

# submission.sentiment = test_classes.astype('int64')

# submission.sentiment.value_counts(normalize=True)

# submission.to_csv('submission7.csv',index=False)

# LSTM

model_LSTM.add(LSTM(64,dropout=0.4,return_sequences=True))
model_LSTM.add(LSTM(32,dropout=0.5,return_sequences=False))
batch_size=16,epochs=10
Train F1 Score : 0.98
Valid1 F1 Score : 0.57
Valid2 F1 Score : 0.6
<br><br>
model_LSTM.add(LSTM(64,dropout=0.9,return_sequences=True))
model_LSTM.add(LSTM(32,dropout=0.9,return_sequences=False))
batch_size=16,epochs=10
Train F1 Score : 0.88
Valid1 F1 Score : 0.63
Valid2 F1 Score : 0.67
<br><br>
model_LSTM.add(LSTM(64,dropout=0.9,return_sequences=True))
model_LSTM.add(LSTM(32,dropout=0.9,return_sequences=False))
batch_size=8,epochs=10
Train F1 Score : 0.88
Valid1 F1 Score : 0.59
Valid2 F1 Score : 0.64
<br><br>
model_LSTM.add(LSTM(32,dropout=0.9,return_sequences=True))
model_LSTM.add(LSTM(16,dropout=0.9,return_sequences=False))
batch_size=16,epochs=10
Train F1 Score : 0.86
Valid1 F1 Score : 0.62
Valid2 F1 Score : 0.64

In [61]:
model_LSTM=Sequential()
model_LSTM.add(Embedding(max_num_words,100,mask_zero=True))
model_LSTM.add(LSTM(32,dropout=0.9,return_sequences=False))
model_LSTM.add(Dense(1,activation='sigmoid'))
model_LSTM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 33        
Total params: 1,017,057
Trainable params: 1,017,057
Non-trainable params: 0
_________________________________________________________________


In [62]:
model_LSTM.compile(loss='binary_crossentropy',optimizer=Adam(lr = 0.001),metrics=[F1_score])
model_LSTM.fit(X_train, y_train,batch_size=16,
                            epochs=10,
                            validation_data=(X_valid1, y_valid1),
                            verbose=2)

Train on 2100 samples, validate on 900 samples
Epoch 1/10
 - 9s - loss: 0.6152 - F1_score: 0.0178 - val_loss: 0.5471 - val_F1_score: 0.0000e+00
Epoch 2/10
 - 7s - loss: 0.4929 - F1_score: 0.2367 - val_loss: 0.4389 - val_F1_score: 0.4361
Epoch 3/10
 - 8s - loss: 0.4097 - F1_score: 0.5728 - val_loss: 0.4037 - val_F1_score: 0.5188
Epoch 4/10
 - 8s - loss: 0.3588 - F1_score: 0.6667 - val_loss: 0.3880 - val_F1_score: 0.5933
Epoch 5/10
 - 7s - loss: 0.3204 - F1_score: 0.6795 - val_loss: 0.3992 - val_F1_score: 0.5954
Epoch 6/10
 - 7s - loss: 0.2927 - F1_score: 0.7313 - val_loss: 0.3996 - val_F1_score: 0.6122
Epoch 7/10
 - 8s - loss: 0.2571 - F1_score: 0.7667 - val_loss: 0.4283 - val_F1_score: 0.5702
Epoch 8/10
 - 9s - loss: 0.2268 - F1_score: 0.8081 - val_loss: 0.4346 - val_F1_score: 0.6261
Epoch 9/10
 - 9s - loss: 0.2024 - F1_score: 0.8307 - val_loss: 0.4750 - val_F1_score: 0.5894
Epoch 10/10
 - 8s - loss: 0.1811 - F1_score: 0.8377 - val_loss: 0.4698 - val_F1_score: 0.6218


In [63]:
train_classes = model_LSTM.predict_classes(X_train)
valid1_classes = model_LSTM.predict_classes(X_valid1)
valid2_classes = model_LSTM.predict_classes(X_valid2)

In [64]:
print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

Train F1 Score : 0.92
Valid1 F1 Score : 0.67
Valid2 F1 Score : 0.67


In [65]:
# Building an LSTM model
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim = max_num_words, 
                    input_length = seq_len, 
                    output_dim = embedding_size)) # Add an embedding layer which represents each unique token as a vector
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(25, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))

model.add(Dense(1, activation='sigmoid'))

In [66]:
from keras.optimizers import Adam
adam = Adam(lr=0.001)

In [67]:
# Mention the optimizer, Loss function and metrics to be computed
model.compile(optimizer=adam,                  # 'Adam' is a variant of gradient descent technique
              loss='binary_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=[F1_score])            # These metrics are computed for evaluating and stored in history

model.fit(X_train, y_train, epochs=10, validation_data=(X_valid1, y_valid1),verbose=2)

Train on 2100 samples, validate on 900 samples
Epoch 1/10
 - 11s - loss: 0.5647 - F1_score: 0.0645 - val_loss: 0.5088 - val_F1_score: 0.3051
Epoch 2/10
 - 7s - loss: 0.3980 - F1_score: 0.6086 - val_loss: 0.4320 - val_F1_score: 0.5825
Epoch 3/10
 - 7s - loss: 0.2517 - F1_score: 0.8018 - val_loss: 0.4739 - val_F1_score: 0.5784
Epoch 4/10
 - 7s - loss: 0.1827 - F1_score: 0.8730 - val_loss: 0.5672 - val_F1_score: 0.5299
Epoch 5/10
 - 8s - loss: 0.1335 - F1_score: 0.9188 - val_loss: 0.5662 - val_F1_score: 0.5680
Epoch 6/10
 - 7s - loss: 0.1065 - F1_score: 0.9358 - val_loss: 0.6826 - val_F1_score: 0.5747
Epoch 7/10
 - 7s - loss: 0.0885 - F1_score: 0.9457 - val_loss: 0.6629 - val_F1_score: 0.5843
Epoch 8/10
 - 7s - loss: 0.0728 - F1_score: 0.9553 - val_loss: 0.7831 - val_F1_score: 0.5528
Epoch 9/10
 - 8s - loss: 0.0772 - F1_score: 0.9495 - val_loss: 0.7904 - val_F1_score: 0.5370
Epoch 10/10
 - 8s - loss: 0.0595 - F1_score: 0.9596 - val_loss: 0.7822 - val_F1_score: 0.5910


In [68]:
train_classes = model.predict_classes(X_train)
valid1_classes = model.predict_classes(X_valid1)
valid2_classes = model.predict_classes(X_valid2)

In [69]:
print('Train F1 Score :',round(f1_score(y_train,train_classes),2))
print('Valid1 F1 Score :',round(f1_score(y_valid1,valid1_classes),2))
print('Valid2 F1 Score :',round(f1_score(y_valid2,valid2_classes),2))

Train F1 Score : 0.98
Valid1 F1 Score : 0.61
Valid2 F1 Score : 0.63
